## Lie Algebra Test

In [68]:
from liegroups import SO3,SE3
import numpy as np

$$
SE(3) = \left\{ \mathbf{T}=
                \begin{bmatrix}
                    \mathbf{R} & \mathbf{t} \\
                    \mathbf{0}^T & 1
                \end{bmatrix} \in \mathbb{R}^{4 \times 4} ~\middle|~ \mathbf{R} \in SO(3), \mathbf{t} \in \mathbb{R}^3 \right\}\\
\mathfrak{se}(3) = \left\{ \boldsymbol{\Xi} =
        \boldsymbol{\xi}^\wedge \in \mathbb{R}^{4 \times 4} ~\middle|~
         \boldsymbol{\xi}=
            \begin{bmatrix}
                \boldsymbol{\rho} \\ \boldsymbol{\phi}
            \end{bmatrix} \in \mathbb{R}^6, \boldsymbol{\rho} \in \mathbb{R}^3, \boldsymbol{\phi} \in \mathbb{R}^3 \right\}
$$

In [88]:
R = SO3.rotx(0.1)
T = np.eye(4,4)
T[0:3,3] = np.array([1,2,3])
T[0:3,0:3] = R.as_matrix()
print(f"T={T}")

print("--------------")

R2 = SO3.rotx(0.5)
T2 = np.eye(4,4)
T2[0:3,3] = np.array([5,2,1])
T2[0:3,0:3] = R2.as_matrix()
print(f"T2={T2}")

pose1 = SE3.from_matrix(T)
pose2 = SE3.from_matrix(T2)

print(f'pose1 is {pose1}')

T=[[ 1.          0.          0.          1.        ]
 [ 0.          0.99500417 -0.09983342  2.        ]
 [ 0.          0.09983342  0.99500417  3.        ]
 [ 0.          0.          0.          1.        ]]
--------------
T2=[[ 1.          0.          0.          5.        ]
 [ 0.          0.87758256 -0.47942554  2.        ]
 [ 0.          0.47942554  0.87758256  1.        ]
 [ 0.          0.          0.          1.        ]]
pose1 is <liegroups.numpy.se3.SE3Matrix>
| [[ 1.          0.          0.          1.        ]
|  [ 0.          0.99500417 -0.09983342  2.        ]
|  [ 0.          0.09983342  0.99500417  3.        ]
|  [ 0.          0.          0.          1.        ]]


In [89]:
Xi_12 = (pose1.inv().dot(pose2)).log()
W = np.eye(4,4)
W[0,0] = 100
W[1,1] = 100
W[2,2] = 100
W[3,3] = 1

inner = np.trace(np.matmul(np.matmul(SE3.wedge(Xi_12),W),SE3.wedge(Xi_12).transpose()))
print(f'Distance is {np.sqrt(inner)}')

Distance is 7.2148293621016615


In [67]:
inner = np.matmul(Xi_12.transpose(),Xi_12)
print(f'Distance is {np.sqrt(inner)}')

Distance is 2.8284271247461903


In [65]:
(np.matmul(pose1.inv().adjoint(), pose2.adjoint(x))

array([[ 1.,  0.,  0.,  0.,  2.,  0.],
       [ 0.,  1.,  0., -2.,  0., -2.],
       [ 0.,  0.,  1.,  0.,  2.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.]])

In [70]:
R

<liegroups.numpy.so3.SO3Matrix>
| [[ 1.          0.          0.        ]
|  [ 0.          0.99500417 -0.09983342]
|  [ 0.          0.09983342  0.99500417]]

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.99500417, -0.09983342],
       [ 0.        ,  0.09983342,  0.99500417]])